## Transactions

In [1]:
import os
import sqlite3

In [2]:
# Create the connection and disable automatic transactions.
connection = sqlite3.connect("3_transactions.db", isolation_level=None)
cursor = connection.cursor()

#### Create tables

In [ ]:
# Create the table for adresses.
cursor.execute(
    """
    CREATE TABLE Adressen (
        AdresId INTEGER PRIMARY KEY,
        Postcode TEXT NOT NULL,
        Huisnummer INTEGER NOT NULL
    );
    """
)

In [ ]:
# Create the customer table which references the adresses table.
cursor.execute(
    """
    CREATE TABLE Klanten (
        KlantId INTEGER PRIMARY KEY,
        AdresId INTEGER NOT NULL,

        Voornaam TEXT NOT NULL,
        Achternaam TEXT NOT NULL,


        UNIQUE(Voornaam, Achternaam),

        FOREIGN KEY(AdresId) REFERENCES Adressen(AdresId)
    );
    """
)

### Transactions

In [ ]:
# Start a transaction.
cursor.execute("BEGIN")

In [ ]:
# Insert an address.
cursor.execute(
    """
    INSERT INTO Adressen
        (Postcode, Huisnummer)
    VALUES
        ('1111 AA', 1)
    ;
    """
)

In [ ]:
# Verify if anything was inserted using DB Browser.

In [ ]:
# Get the automatically generated address ID.
address_id = cursor.lastrowid
address_id

In [ ]:
# Insert a customer with the generated address ID.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, AdresId)
    VALUES
        ('Ingrid', 'Jansen', ?)
    ;
    """,
    [address_id]
)

In [ ]:
# Commit changes to the database.
# Alternative: db.commit()
cursor.execute("COMMIT")

In [17]:
# Check whether data was inserted using DB Browser.


### Handle errors

In [ ]:
# Define templates for the queries.
address_query = """
    INSERT INTO Adressen
    (Postcode, Huisnummer)
    VALUES (:Postcode, :Huisnummer);
"""

customer_query = """
    INSERT INTO Klanten
        (Voornaam, Achternaam, AdresId)
    VALUES (:Voornaam, :Achternaam, :AdresId);
"""

In [23]:
# Define some dummy data.
customers = [
    {"Voornaam": "Mark", "Achternaam": "Vos"},
    {"Voornaam": "Mark", "Achternaam": "Vos"},
]

addresses = [
    {"Postcode": "2222 BB", "Huisnummer": 2},
    {"Postcode": "3333 CC", "Huisnummer": 3},
]

In [ ]:
# Loop over customers and their adresses.
for customer, address in zip(customers, addresses):

    # Use try ... except to handle errors.
    try:

        # Insert the address first.
        cursor.execute(address_query, address)

        # Get the generated ID and link it to the customer.
        customer["AdresId"] = cursor.lastrowid

        # Then insert the customer.
        cursor.execute(customer_query, customer)

        # Commit the data.
        db.commit()

    except sqlite3.IntegrityError:

        # Roll back if anything goes wrong.
        db.rollback()

### Clean up

In [26]:
cursor.close()
connection.close()
os.remove("3_transactions.db")